# Implementación de Agente Conversacional

En este notebook implementaremos un agente conversacional utilizando servicios de OpenAI.

Este notebook nos guiará por el camino hacia una implementación realmente utilizable en la práctica, pasando por diferentes puntos intermedios de complejidad, y por supuesto de desempeño. Empezaremos con un chat básico, y le iremos agregando características que mejoran las respuestas, aprendiendo a tomar más control sobre las respuestas que el Agente brinda.

A lo largo del notebook, hay algunas tareas de programación para ser implementadas. Haremos pausas en la clase para dar tiempo a pensarlas e implementarlas, para luego revisar la solución entre todos. Los campos, junto con una descripción del ejercicio a realizar, están marcados con el siguiente texto:

```
# ================================================================
# Complete aquí su solución
# ----------------------------------------------------------------




# ================================================================
```

A continuación se presentan las grandes etapas a implementar.

### 1. Agente "Vanila"
Empezaremos creando un agente básico, cuyo único propósito sea llamar a la API de OpenAI, y de ese modo contestar las preguntas y mantener una conversación con el usuario, utilizando únicamente el conocimiento que el modelo de lenguaje ya adquirió durante el pre-entrenamiento

### 2. Agregado de documentos en prompt
Luego veremos la técnica más sencilla para tratar de controlar, al menos un poco, las respuestas del Agente, adentrándonos en el terreno del llamado `prompt engineering`. En este caso pasaremos junto con el prompt, una cierta cantidad de documentos que contengan la información relevante, para que el modelo pueda extraer la respuesta de allí.

### 3. RAG: inclusión de fuentes de información a través de base vectorial
La técnica RAG (Retrieval-Augmented Generation) es una técnica de procesamiento de lenguaje natural que combina modelos de recuperación de información y modelos generativos para mejorar la calidad de las respuestas generadas. En lugar de depender únicamente del conocimiento almacenado en el modelo generativo, RAG incorpora un mecanismo de recuperación que busca información relevante en una base de datos externa o corpus de documentos antes de generar una respuesta.

Para este caso construiremos una base vectorial con unos pocos documentos provistos por Montes del Plata, a modo de ejemplo. Se codificarán en una base de la librería Chroma. Cada vez que llega una pregunta del usuario, el Agente realizará una búsqueda entre dichos documentos para encontrar cuál es el/los documentos que más probablemente contengan la respuesta. Una vez identificados, se extrae el texto de los documentos y se envía junto con la pregunta al modelo de lenguaje natural, de modo que éste extraiga la información de ellos.

### 4. Control de flujo
En esta sección implementaremos una versión sencilla del control de flujo. Para ejemplificarlo, utilizaremos un modelo que implementa reglas de moderación en los mensajes. De este modo, primero se chequea si el mensaje pasa las reglas de moderación. Si no pasa se da por terminado el chat inmediatamente. De lo contrario, se procede al flujo normal de la conversación. 

Si bien el ejemplo es sencillo, es fácilmente extensible a flujos más complejos

### Instalación de librerías
Primero que nada instalamos los paquetes necesarios

In [4]:
# Instalamos los paquetes necesarios
%pip install pysqlite3
%pip install chromadb
%pip install openai

StatementMeta(spAzureML, 178, 16, Finished, Available, Finished)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 KB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for pysqlite3: filename=pysqlite3-0.5.3-cp310-cp310-linux_x86_64.whl size=47515 sha256=f378aac1972b0100fce31c5bbf01d731e418834574f2af5669ddb238fd647667
  Stored in directory: /home/trusted-service-user/.cache/pip/wheels/6d/5b/04/3e6115d6b90cd4d63a55c4d13685fe8117bb87d5997a215e71
Successfully built pysqlite3
You should consider upgrading via the '/nfs4/pyenv-5890e340-53bd-4fe2-a8e6-a223006114e0/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 KB 16.0 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 KB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.0/94.0 KB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━

### Importamos los paquetes a utilizar

Y definimos algunas variables que se utilizarán a lo largo de todo el notebook

In [5]:
from typing import List, Union

from openai import OpenAI
import os
import re
import pysqlite3
import chromadb
import pandas as pd
from chromadb.api.models.Collection import Collection
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction


pd.set_option('max_colwidth', 1000)

# Seteamos variables
OPENAI_API_KEY = ''
LLM_MODEL = "gpt-3.5-turbo-0125"

StatementMeta(spAzureML, 178, 22, Finished, Available, Finished)

# 1) Versión 1 del agente

Utilizando solo el conocimiento que tiene el modelo de lenguaje, y utilizando un prompt para guiar la respuesta

La idea es crear una clase `Chatbot`, para dejar encapsulado y prolijo todo lo referente a la interacción con el modelo de lenguaje natural.

Tiene un prompt "oculto", que se le pasa con la `key` "system". Es una instrucción general para que el modelo responda. Esta técnica es tal vez la más básica para empezar a controlar las respuestas del modelo. 

Es más bien una "sugerencia", ya que no estamos imponiendo ninguna restricción dura, pero efectivamente genera un cambio en las respuestas

## VanilaChatBot

Le llamamos "Vanila" al chatbot más básico posible. 
A medida que avancemos en este notebook iremos complejizándolo.

El procedimiento que seguiremos es el siguiente:

1. Definir la clase
2. Crear una instancia
3. Hacer una pregunta

In [6]:
# 1. Definimos la clase 

class VanilaChatBot:
    def __init__(self, prompt: Union[str, None] = None, llm_name: Union[str, None] = None):
        
        # Establecemos conexión con OpenAI
        self.openai_client = OpenAI(api_key=OPENAI_API_KEY)
        self.total_cost = 0

        if prompt:
            self.history: List[str] = [{'role':'system', 'content': prompt}]
        else:
            self.history: List[str] = [{
                'role':'system', 
                'content': "Te van a hacer preguntas acerca del proceso de Transporte de madera. " 
                    "Contestar con información técnica que tengas en tu base de conocimientos."
            }]
        
        if llm_name:
            self.llm_name = llm_name
        else:
            self.llm_name = "gpt-3.5-turbo-0125"
    
    def chat(self, message: str, print_conversation: bool = True):
        self.process_new_message(message, print_conversation)

    def process_new_message(self, message: str, print_conversation: bool = True):
        self.history.append({'role':'user', 'content':message})
        
        response=self.get_response()
        self.history.append({'role':'assistant', 'content':response})
        
        if print_conversation:
            self.print_history()

    def get_response(self, temperature=0):
        response = self.openai_client.chat.completions.create(
            model=self.llm_name,
            messages=self.history,
            temperature=temperature,
        )
        self.total_cost += self.openai_api_calculate_cost(usage=response.usage.to_dict())
        return response.choices[0].message.content

    def print_history(self):
        def bold(text: str):
            return '\033[1m'+ text +'\033[0m'

        for message in self.history[1:]:
            print(f"{bold(message['role'].upper())}: {message['content']}\n")
        
        print(f"\n {bold('Costo total de la conversación'.upper())}: {self.total_cost:.7f}")

    def openai_api_calculate_cost(self, usage: dict) -> float:
        pricing = {
            'gpt-3.5-turbo-0125': {
                'prompt': 0.5,
                'completion': 1.5,
            },
            'gpt-4': {
                'prompt': 30,
                'completion': 60,
            },
            'gpt-4-0125-preview': {
                'prompt': 10,
                'completion': 30,
            },
            'gpt-4o': {
                'prompt': 5,
                'completion': 15,
            },
            'text-embedding-3-small': {
                'prompt': 0.02,
                'completion': 0.02,
            },
            'text-embedding-3-large': {
                'prompt': 0.13,
                'completion': 0.13,
            }
        }

        try:
            model_pricing = pricing[self.llm_name]
        except KeyError:
            raise ValueError("Invalid model specified")

        prompt_cost = usage['prompt_tokens'] * model_pricing['prompt'] / 1000000
        completion_cost = usage['completion_tokens'] * model_pricing['completion'] / 1000000

        total_cost = prompt_cost + completion_cost
        return total_cost

StatementMeta(spAzureML, 178, 23, Finished, Available, Finished)

In [7]:
# 2. Creamos una instancia
vanila_chatbot = VanilaChatBot(llm_name=LLM_MODEL)

StatementMeta(spAzureML, 178, 24, Finished, Available, Finished)

In [8]:
# 3. Hacemos una pregunta
vanila_chatbot.chat(""" Existe alguna reglamentación que trate el transporte de la madera? \
Responder de forma concisa y solo indicando la identificación de las reglamentaciones.
""")

StatementMeta(spAzureML, 178, 25, Finished, Available, Finished)

USER:  Existe alguna reglamentación que trate el transporte de la madera? Responder de forma concisa y solo indicando la identificación de las reglamentaciones.


ASSISTANT: Sí, algunas de las reglamentaciones que tratan el transporte de la madera son:

1. Norma NCh 2369 Of.2003 Transporte de madera en rollo - Requisitos generales.
2. Reglamento de Transporte Terrestre de Materiales y Residuos Peligrosos (Decreto Supremo N° 148/2003 del Ministerio de Transportes y Telecomunicaciones de Chile).
3. Reglamento de Transporte de Carga Peligrosa por Carretera (Resolución N° 1.134/2003 de la Comisión Nacional de Energía de Chile).


 COSTO TOTAL DE LA CONVERSACIÓN: 0.0002505


In [9]:
# Continuamos con la conversación
vanila_chatbot.chat("De donde son estas reglamentaciones?")

StatementMeta(spAzureML, 178, 26, Finished, Available, Finished)

USER:  Existe alguna reglamentación que trate el transporte de la madera? Responder de forma concisa y solo indicando la identificación de las reglamentaciones.


ASSISTANT: Sí, algunas de las reglamentaciones que tratan el transporte de la madera son:

1. Norma NCh 2369 Of.2003 Transporte de madera en rollo - Requisitos generales.
2. Reglamento de Transporte Terrestre de Materiales y Residuos Peligrosos (Decreto Supremo N° 148/2003 del Ministerio de Transportes y Telecomunicaciones de Chile).
3. Reglamento de Transporte de Carga Peligrosa por Carretera (Resolución N° 1.134/2003 de la Comisión Nacional de Energía de Chile).

USER: De donde son estas reglamentaciones?

ASSISTANT: Las reglamentaciones mencionadas son de Chile.


 COSTO TOTAL DE LA CONVERSACIÓN: 0.0003875


## Ejercicio

Cambie el prompt del sistema para ver una respuesta distinta.

Lineamiento: No cambiar el código de la clase `VanilaChatBot`

In [ ]:
# ================================================================
# Complete aquí su solución
# ----------------------------------------------------------------

# Genere un nuevo prompt


# Cree una nueva instancia de la clase VanilaChatBot que use ese prompt


# Chatee con el modelo



# ================================================================

StatementMeta(, , , Waiting, , Waiting)

In [ ]:
# Solución 

vanila_chatbot_random = VanilaChatBot(
    prompt="Responde con una analogía sofisticada de fútbol.",
    llm_name=LLM_MODEL
)

vanila_chatbot_random.chat("Existe alguna reglamentación que trate el transporte de la madera?")

StatementMeta(, , , Waiting, , Waiting)

# 2) Agregado de documentos en prompt

En esta sección veremos la técnica más sencilla posible para hacer que el modelo de lenguaje responda en base a ciertos documentos con información relevante para el caso de uso. 

Utilizaremos algunos documentos provistos por Montes del Plata. 

Típicamente queremos hacer esto cuando tenemos documentos con información particular de nuestro problema, y dicha información probablemente no se encontrará disponible en la base de conocimiento de los modelos de lenguaje pre-entrenados.

Para esta prueba de concepto utilizaremos 10 documentos, que están guardados en la carpeta `capacitacion-digitalsense/clase3/documents`.

La metodología a utilizar en este caso es simplemente pasar el texto como parte del prompt.



In [10]:
# Cargamos los documentos a usar
directory = "capacitacion-digitalsense/clase3/documents"
documents_list = ['document_0.txt', 'document_1.txt', 'document_2.txt', 'document_3.txt', 'document_4.txt', 'document_5.txt', 
'document_6.txt', 'document_7.txt', 'document_8.txt', 'document_9.txt', 'document_10.txt', 'document_11.txt']

documents_dict = {}
for doc in documents_list:
    file_path = os.path.join(directory, doc)
    with open(file_path, "r") as my_file:
        documents_dict[doc[:-4]] = my_file.read()

StatementMeta(spAzureML, 178, 27, Finished, Available, Finished)

In [11]:
# Instrucción inicial
prompt = "Te van a dar información técnica acerca de un proceso y te van a hacer preguntas acerca de esa información. "\
    "Debes responder solo en base a la información que te proporcionaron. "\
    "En caso que te hagan una pregunta cuya respuesta no quede comprendida en la información provista, "\
    "responder que no contas con información para responder la pregunta. "\
    "Responder siempre en español. "\
    "Documentos: "

# Agregamos todos los documentos al prompt
for i, doc in enumerate(documents_dict.values()):
    prompt += "Capítulo "+str(i)+": " + doc

# Visualizamos el prompt
print(prompt)

StatementMeta(spAzureML, 178, 28, Finished, Available, Finished)

Te van a dar información técnica acerca de un proceso y te van a hacer preguntas acerca de esa información. Debes responder solo en base a la información que te proporcionaron. En caso que te hagan una pregunta cuya respuesta no quede comprendida en la información provista, responder que no contas con información para responder la pregunta. Responder siempre en español. Documentos: Capítulo 0: 
P_LOGT_01: TRANSPORTE DE MADERA 
VERSIÓN: 02
27/10/2023

ÍNDICE
1.	OBJETIVO
2.	ALCANCE
3.	LINEAMIENTOS GENERALES
3.1	Requerimientos documentales	
3.2	Empresa Transportista
3.3	Requerimientos contractuales
4.	DESCRIPCIÓN DEL TRANSPORTE DE MADERA
4.1	Planificación
4.2	Habilitación del conductor y vehículo
4.3	Carga
4.4	Acondicionamiento de la carga
4.5	Tiempo de conducción
4.6	Generación de la Orden de Transporte
4.7	Remitos
5.	SEGUIMIENTO Y CONTROL
5.1	Operaciones
5.2	Incumplimientos
5.3	Control de alcohol y drogas
5.4	Velocidad
5.5	Criterios de detención inmediata
6.	RÉGIMEN DE SANCIONES
7.	EMER

In [12]:
# Consultamos al modelo
vanila_chatbot_with_documents = VanilaChatBot(prompt)
vanila_chatbot_with_documents.chat(
    "Que debe hacer el transportista en caso que detecte que las luces delanteras del vehículo no funcionan?"
)

StatementMeta(spAzureML, 178, 29, Finished, Available, Finished)

USER: Que debe hacer el transportista en caso que detecte que las luces delanteras del vehículo no funcionan?

ASSISTANT: En caso de que el transportista detecte que las luces delanteras del vehículo no funcionan, debe proceder de la siguiente manera:

- Si la detección se realiza en horario diurno, debe repararlas antes de que oscurezca o antes del próximo viaje.
- En caso de que la detección se realice en horario nocturno, se prohíbe circular y se debe reparar antes de continuar el viaje. No está permitido circular con balizas, solo hasta encontrar un lugar seguro para detenerse.


 COSTO TOTAL DE LA CONVERSACIÓN: 0.0045070


# 3) RAG

Retrieval-Augmented Generation (RAG) es una técnica de procesamiento de lenguaje natural que combina modelos de recuperación de información y modelos generativos para mejorar la calidad de las respuestas generadas. En lugar de depender únicamente del conocimiento almacenado en el modelo generativo, RAG incorpora un mecanismo de recuperación que busca información relevante en una base de datos externa o corpus de documentos antes de generar una respuesta.

En un flujo típico de RAG, cuando se recibe una consulta, el sistema primero recupera documentos o fragmentos relevantes del corpus. Luego, estos fragmentos se combinan con la consulta original y se pasan a un modelo generativo, como GPT, para generar una respuesta más informada y precisa. Esta combinación permite al modelo aprovechar información actualizada y específica, que puede no estar incluida en el entrenamiento del modelo generativo original. RAG es útil para tareas donde la precisión y la relevancia de la información son cruciales, como la respuesta a preguntas, el resumen de documentos y la generación de textos basados en datos externos.

## ChromaDatabase

En esta sección comenzaremos analizando la base de datos disponible y veremos los métodos existentes para la recuperación de documentos.

Luego realizaremos un ejercicio donde se deberán hacer distintas consultas a la base de datos.

In [13]:
class ChromaDatabase:
    def __init__(self):
        
        # Establecemos conexión con la base de datos
        self.client_chroma = chromadb.PersistentClient(path="capacitacion-digitalsense/clase4/chroma_db")
        # Definimos el modelo de embedding a utilizar
        embedding_model = "text-embedding-3-small"
        self.embedding_function = OpenAIEmbeddingFunction(api_key=OPENAI_API_KEY, model_name=embedding_model)

    # Método para seleccionar la colección a utilizar
    def get_collection(self, collection_name: str) -> Collection:
        return self.client_chroma.get_collection(name=collection_name, embedding_function=self.embedding_function)
    
    def db_search(
        self,
        collection_name: Union[str, None] = None,
        query: Union[str, None] = None,
        max_results: int = 3,
        condition: Union[dict, None] = None,
        where_document: Union[dict, None] = None,
    ) -> pd.DataFrame:
        
        if collection_name:
            self.collection_name = collection_name
        else:
            self.collection_name = "Montes_del_Plata"
        
        collection = self.get_collection(self.collection_name)

        if query:
            df = self._check_and_pass(
                    self._query_collection, 
                    query=query, 
                    collection=collection, 
                    max_results=max_results, 
                    condition=condition, 
                    where_document=where_document
                )
        else:
            df = self._check_and_pass(
                    self._get_from_collection,
                    collection=collection,
                    condition=condition, 
                    where_document=where_document
                )

        return df
    
    # Método para recuperar documentos a partir de una consulta
    def _query_collection(
        self, 
        query: str,
        collection: chromadb.Collection, 
        max_results: int = 3,
        condition: Union[dict, None] = None,
        where_document: Union[dict, None] = None,
    ) -> pd.DataFrame:
        
        if condition and where_document:
            results = collection.query(query_texts=query, n_results=max_results, 
                                       where=condition, where_document=where_document,
                                       include=["distances", "documents", "metadatas"])
        elif condition and not where_document:
            results = collection.query(query_texts=query, n_results=max_results, 
                                       where=condition, include=["distances", "documents", "metadatas"])
        elif where_document and not condition:
            results = collection.query(query_texts=query, n_results=max_results, where_document=where_document, 
                                       include=["distances", "documents", "metadatas"])
        else:
            results = collection.query(query_texts=query, n_results=max_results,
                                       include=["distances", "documents", "metadatas"])
        
        df = self._generate_df_from_results_for_query(results)
        return df
    
    # Método para recuperar documentos a partir de palabras contenidas en el texto
    def _get_from_collection(
        self,
        collection: chromadb.Collection, 
        condition: Union[dict, None] = None,
        where_document: Union[dict, None] = None,
    ) -> pd.DataFrame:
        
        if condition and where_document:
            results = collection.get(where=condition, where_document=where_document,
                                    include=["documents", "metadatas"])
        elif condition and not where_document:
            results = collection.get(where=condition, include=["documents", "metadatas"])
        elif where_document and not condition:
            results = collection.get(where_document=where_document, include=["documents", "metadatas"])
        df = self._generate_df_from_results_for_get(results)
        return df

    def _check_and_pass(self, func, **kwargs):
        # Filtra los argumentos con valor None y llama a la función func con el resto de argumentos
        filtered_kwargs = {k: v for k, v in kwargs.items() if v is not None}
        return func(**filtered_kwargs)

    def _generate_df_from_results_for_query(self, results) -> pd.DataFrame:
        df = pd.DataFrame({
                "id"               : results["ids"][0],
                "distancia"        : results["distances"][0],
                "contenido"        : [doc for doc in results["documents"][0]],
                "departamento"     : [dict["departamento"] for dict in results["metadatas"][0]],
                "nombre_documento" : [dict["nombre_documento"] for dict in results["metadatas"][0]],
                "capitulo"         : [dict["capitulo"] for dict in results["metadatas"][0]]
                })
        return df
    
    def _generate_df_from_results_for_get(self, results) -> pd.DataFrame:
        df = pd.DataFrame({
                "id"               : results["ids"],
                "contenido"        : [doc for doc in results["documents"]],
                "departamento"     : [dict["departamento"] for dict in results["metadatas"]],
                "nombre_documento" : [dict["nombre_documento"] for dict in results["metadatas"]],
                "capitulo"         : [dict["capitulo"] for dict in results["metadatas"]]
                })
        return df

StatementMeta(spAzureML, 178, 30, Finished, Available, Finished)

In [14]:
# Creamos una instancia de la clase
chroma_db = ChromaDatabase()

# Chequeamos las colecciones que tiene disponible la base de datos
chroma_db.client_chroma.list_collections()

StatementMeta(spAzureML, 178, 31, Finished, Available, Finished)

[Collection(id=809de891-0087-4da8-8313-738b1ba24ff7, name=TRANSPORTE_DE_MADERA),
 Collection(id=b0407bae-45f3-4f24-aa16-f56f8a8c649e, name=Montes_del_Plata)]

In [15]:
# Seleccionamos la colección con la que queremos trabajar
collection_name="Montes_del_Plata"
mdp_collection = chroma_db.get_collection(collection_name=collection_name)

# Observamos la cantidad de datos que tiene la colección
print(f"Esta colección contiene {mdp_collection.count()} documentos.")

StatementMeta(spAzureML, 178, 32, Finished, Available, Finished)

Esta colección contiene 33 documentos.


## Ejercicio

Realice una búsqueda en la base de datos para cada una de las siguientes consignas y muestre los resultados:

1) La búsqueda se genera a partir de una consulta.

2) La búsqueda se genera a partir de una consulta y además la metadata cumple una condición.

3) Se deben buscar todos los documentos de la base de datos que contengan una palabra clave.

In [16]:
# Ejemplo de condiciones
consulta = "Que debe hacer el transportista en caso que detecte que las luces delanteras del vehículo no funcionan?"

condicion = {
    "departamento": {
        "$eq": "Forestal"
    }
}

buscar_documento = {
    "$contains": "blanqueo"
}

StatementMeta(spAzureML, 178, 33, Finished, Available, Finished)

In [17]:
# Solución 1
df_consigna_uno = chroma_db.db_search(query=consulta, collection_name=collection_name)
df_consigna_uno

StatementMeta(spAzureML, 178, 34, Finished, Available, Finished)

,id,distancia,contenido,departamento,nombre_documento,capitulo
0,5,0.430981,"\n5. SEGUIMIENTO Y CONTROL\n5.1 Operaciones\nEl control de las operaciones es responsabilidad del contratista. MdP realiza un control adicional por parte del Encargado y/o Supervisor de Transporte, con apoyo del Técnico SySO de MdP y por servicios contratados para tal fin, ya sea en la ruta o en los puntos de origen y destino de la carga. La información recabada se ingresa en el registro respectivo (ej. Datacenter), para procesamiento de datos y generación de estadísticas, indicadores de desempeño e historiales de cumplimiento.\n\n5.2 Incumplimientos\nCon el fin de trabajar en la gestión de la seguridad del transporte, se definen las siguientes penalizaciones por incumplimientos del transportista en materia de seguridad:\nPor cada omisión de reportar/informar un Accidente con Tiempo Perdido (ACTP) a la Central de Transporte de MdP en el plazo establecido en el Procedimiento de Investigación de Accidentes de MdP, el Transportista será pasible de una multa de hasta USD 3.000 por omis...",Forestal,P_LOGT_01: TRANSPORTE DE MADERA,5
1,3,0.488050,"\n3. LINEAMIENTOS GENERALES\nEl área de Transporte planifica su actividad según los lineamientos establecidos en el P_Plan_01: Planeamiento Forestal:\nPlan Estratégico de Transporte (PE) a largo plazo\nPlan Táctico de Transporte (5 años)\nPlan Operativo Mensual de Transporte (POA)\n\n3.1 Requerimientos documentales\nPara poder realizar el servicio de transporte de madera, las empresas contratadas deben presentar documentación a MdP a fin de realizar los controles de la documentación legal laboral y registros del transportista, y la evaluación respectiva del mismo, asegurando el cumplimiento de las normativas legales y contractuales que le competen (ver también P_RH_03 Contratación y Evaluación de Contratistas Forestales), de acuerdo con los siguientes criterios:\n\nTransportista: Empresa\nDocumentos:\n- Certificado de empresa profesional de carga\n- BPS: Certificados, recibos y consulta de actividad por empresa al día\n- Certificado de DGI vigente\n- Seguro y certificados al día (B...",Forestal,P_LOGT_01: TRANSPORTE DE MADERA,3
2,9,0.528077,"\n9. RESPONSABILIDADES\nResponsables: Responsable Recursos Humanos o Responsable asignado\nAcciones: Revisión de documentación legal (registros empresa transportista, legal-laboral, de inspección vehicular y circulación vial).\nResponsables: Supervisor Transporte MdP \nAcciones: Controlar las operaciones de transporte de madera\nResponsables: Encargado de operaciones de transporte MdP\nAcciones: Verificar el correcto funcionamiento de las unidades de transporte, el acondicionamiento de carga y las operaciones en campo.\nResponsables: Técnico SySO\nAcciones:\nBrindar apoyo en el control de las actividades de carga, acondicionamiento de carga y elementos de seguridad de las unidades de transporte.\nInvestigar causas de accidentes.\n",Forestal,P_LOGT_01: TRANSPORTE DE MADERA,9


In [18]:
# Solución 2
df_consigna_dos = chroma_db.db_search(query=consulta, collection_name=collection_name, condition=condicion)
df_consigna_dos

StatementMeta(spAzureML, 178, 35, Finished, Available, Finished)

,id,distancia,contenido,departamento,nombre_documento,capitulo
0,5,0.430981,"\n5. SEGUIMIENTO Y CONTROL\n5.1 Operaciones\nEl control de las operaciones es responsabilidad del contratista. MdP realiza un control adicional por parte del Encargado y/o Supervisor de Transporte, con apoyo del Técnico SySO de MdP y por servicios contratados para tal fin, ya sea en la ruta o en los puntos de origen y destino de la carga. La información recabada se ingresa en el registro respectivo (ej. Datacenter), para procesamiento de datos y generación de estadísticas, indicadores de desempeño e historiales de cumplimiento.\n\n5.2 Incumplimientos\nCon el fin de trabajar en la gestión de la seguridad del transporte, se definen las siguientes penalizaciones por incumplimientos del transportista en materia de seguridad:\nPor cada omisión de reportar/informar un Accidente con Tiempo Perdido (ACTP) a la Central de Transporte de MdP en el plazo establecido en el Procedimiento de Investigación de Accidentes de MdP, el Transportista será pasible de una multa de hasta USD 3.000 por omis...",Forestal,P_LOGT_01: TRANSPORTE DE MADERA,5
1,3,0.488050,"\n3. LINEAMIENTOS GENERALES\nEl área de Transporte planifica su actividad según los lineamientos establecidos en el P_Plan_01: Planeamiento Forestal:\nPlan Estratégico de Transporte (PE) a largo plazo\nPlan Táctico de Transporte (5 años)\nPlan Operativo Mensual de Transporte (POA)\n\n3.1 Requerimientos documentales\nPara poder realizar el servicio de transporte de madera, las empresas contratadas deben presentar documentación a MdP a fin de realizar los controles de la documentación legal laboral y registros del transportista, y la evaluación respectiva del mismo, asegurando el cumplimiento de las normativas legales y contractuales que le competen (ver también P_RH_03 Contratación y Evaluación de Contratistas Forestales), de acuerdo con los siguientes criterios:\n\nTransportista: Empresa\nDocumentos:\n- Certificado de empresa profesional de carga\n- BPS: Certificados, recibos y consulta de actividad por empresa al día\n- Certificado de DGI vigente\n- Seguro y certificados al día (B...",Forestal,P_LOGT_01: TRANSPORTE DE MADERA,3
2,9,0.528077,"\n9. RESPONSABILIDADES\nResponsables: Responsable Recursos Humanos o Responsable asignado\nAcciones: Revisión de documentación legal (registros empresa transportista, legal-laboral, de inspección vehicular y circulación vial).\nResponsables: Supervisor Transporte MdP \nAcciones: Controlar las operaciones de transporte de madera\nResponsables: Encargado de operaciones de transporte MdP\nAcciones: Verificar el correcto funcionamiento de las unidades de transporte, el acondicionamiento de carga y las operaciones en campo.\nResponsables: Técnico SySO\nAcciones:\nBrindar apoyo en el control de las actividades de carga, acondicionamiento de carga y elementos de seguridad de las unidades de transporte.\nInvestigar causas de accidentes.\n",Forestal,P_LOGT_01: TRANSPORTE DE MADERA,9


In [19]:
# Solución 3
df_consigna_tres = chroma_db.db_search(collection_name=collection_name, where_document=buscar_documento)
df_consigna_tres

StatementMeta(spAzureML, 178, 36, Finished, Available, Finished)

,id,contenido,departamento,nombre_documento,capitulo
0,12,\nInstrucciones de operación\nde procesos\nBlanqueo\nMontes del Plata\nReferencia D-02-811373-025\nEstado: FINAL\nFecha: 08-02-2013\nAutor: N Littman\n\nDocumento confidencial. Reservados todos los derechos. No se permiten la duplicación ni la entrega a terceros sin la autorización escrita de ANDRITZ AG.\n\nCliente: Montes del Plata\nNombre de la planta: Montes del Plata\n\nContacto: ANDRITZ OY\nDivisión: División de tecnologías de fibra \n\nResponsable de proyecto: Mikko Metso\n\nTeléfono: +358 (0)20 450 5555\nFax: +358 (0)20 450 5540\nCorreo electrónico: fiberline@andritz.com\nwww.andritz.com\n\nÍNDICE\n\n1.\tGENERALIDADES\t4\n2.\tDATOS TÉCNICOS\t5\n2.1\tEspecificaciones de las máquinas principales\t5\n2.2\tDatos de operación del proceso principal\t5\n2.3\tEquipos principales del proceso\t6\n2.3.1\tRaspador de descarga de la torre de almacenamiento de pulpa sin blanquear\t6\n2.3.2\tBombeo MC hacia la etapa Da\t7\n2.3.3\tBombeo MC a otras etapas de blanqueo\t9\n2.3.4\tMezcladores...,Planta,Instrucciones de operación de procesos: Blanqueo,0
1,13,"\n1. GENERALIDADES\n\nEste documento describe la operación del departamento de blanqueo. Para una operación segura y óptima es necesario seguir estas instrucciones. Los usuarios del proceso deben familiarizarse también con los siguientes documentos:\n\nDescripciones de procesos (documentos 1000-153-P40-105207)\nPlanos funcionales (documentos 1000-153-I50-107080, 1000-153-I50-107081, 1000-153-I50-107082, 1000-153-I50-107083, 1000-153-I50-107084, 1000-153-I50-111589)\nDiagramas de procesos e instrumentación (documentos 1000-153-P20-101450, 1000-153-P20-101451, 1000-153-P20-101452, 1000-153-P20-101453, 1000-153-P20-101454, 1000-153-P20-101455)\nPlanos de disposición\n\nEn este documento, los equipos y conexiones del proceso se describen tal y como se utilizarán durante la puesta en marcha de la planta. Este manual de operación tiene como fin servir como material de capacitación para los operadores antes de la puesta en marcha. Más adelante, puede usarse como una introducción para nuev...",Planta,Instrucciones de operación de procesos: Blanqueo,1
2,14,"\n2. DATOS TÉCNICOS\n\nLas siguientes instrucciones de proceso describen el proceso de blanqueo Da-EP-D1-P, que forma parte del proceso de pulpado de la línea de fibra. El blanqueo en este documento comprende un área que comienza en el torre de almacenamiento de pulpa sin blanquear (153-21-001) y termina con el bombeo hacia la torre HD de pulpa blanqueada, 153-21-503. La depuración precede al blanqueo. Los procesos posteriores son el secado de la pulpa o la fabricación del papel. El lavador de gases de la planta de blanqueo también se describe en este documento.\n\nEsta descripción contiene información detallada de ingeniería y se usa junto con los diagramas de procesos e instrumentación y los planos funcionales para la ingeniería de control de procesos y para la configuración del sistema DCS de la planta.\n\n2.1 Especificaciones de las máquinas principales\n\nLas máquinas principales se especifican en las listas de equipos separadas.\n\n2.2 Datos de operación del proceso principal...",Planta,Instrucciones de operación de procesos: Blanqueo,2
3,15,"\n3. PUESTA EN MARCHA\n\n3.1 Operaciones antes de la puesta en marcha\n\n3.1.1 Generalidades\n\nEn este capítulo se definen las tareas que es necesario realizar antes de la puesta en marcha. Estas tareas preparatorias pueden y deben hacerse con la antelación suficiente para garantizar una puesta en marcha segura y protegida. Es necesario comprobar la situación mecánica, la electrificación y la instrumentación de la planta, así como garantizar la disponibilidad de los suministros. \n\nTras las tareas preparatorias, el departamento de blanqueo está preparado para ponerse en marcha de acuerdo con las condiciones de enclavamiento.\n\n3.1.2 Comprobaciones mecánicas\n\nAntes de la puesta en marcha, es necesario comprobar lo siguiente:\nTodos los

## RAGChatBot

Crearemos aquí una clase `RAGChatBot`, que herede de `VanilaChatBot`, para poder reutilizar la mayor parte del código.

### Ejercicio
Armar la sentencia para efectivamente hacer la llamada al modelo, pasando los documentos más cercanos a la pregunta.

In [20]:
class RAGChatBot(VanilaChatBot):
    def __init__(self, prompt: Union[str, None] = None, llm_name: Union[str, None] = None):
        super(RAGChatBot, self).__init__(prompt=prompt, llm_name=llm_name)
        
        self.chroma_db = ChromaDatabase()
        self.db_search_done = False

        if prompt:
            self.history: List[str] = [{'role':'system', 'content': prompt}]
        else:
            self.history: List[str] = [{
                'role':'system', 
                'content': "Te harán preguntas técnicas y te proporcionarán información específica "\
                    "para que puedas responder basándote en esa información. "\
                    "En caso que te hagan una pregunta cuya respuesta no quede comprendida en la información provista, "\
                    "responder que no contas con información para responder la pregunta. "\
                    "Responder siempre en español. "\
                    "Al responder indicar de donde extrajiste la información para elaborar la respuesta. "\
            }]

    # Método donde se arma el mensaje con todos los documentos para consultarle al modelo   
    def prepare_message(self, message: str) -> str:

        # Primero buscamos en la base de datos los documentos más cercanos a la consulta del usuario
        df_chroma_results = self.chroma_db.db_search(query=message)
        self.db_search_done = True
        
        # Armamos la consulta a realizar al modelo
        query = "La pregunta del usuario es: '" + message + "'.\n La información para responder la pregunta es"\
        " la que se indica a continuación: \n"
        self.query_header = query

        # Agregamos documentos al mensaje
        for i in range(len(df_chroma_results)):
            row = df_chroma_results.iloc[i]
            query += "Capítulo "+row['capitulo']+" del documento " + row['nombre_documento']+": "\
            + row['contenido'] + "\n"
        return query

    # Sobreescribimos la clase chat heredada de VanilaChatBot
    def chat(self, message: str, print_conversation: bool = True):

        # ================================================================
        # Complete aquí su solución
        # ----------------------------------------------------------------
        
        if self.db_search_done:
            self.process_new_message(message, print_conversation)
        else:
            query = self.prepare_message(message)
            self.process_new_message(query, print_conversation)

        
        # ================================================================
    
    # Sobrescribimos el método print_history para no mostrar todo el contenido de los documentos que
    # le estamos pasando al modelo
    def print_history(self):
        def bold(text: str):
            return '\033[1m'+ text +'\033[0m'

        for message in self.history[1:]:
            if message['role'] == 'user' and self.query_header in message['content']:
                message_to_print = self.query_header
                # Regular expression pattern
                pattern = r"Capítulo \d+ del documento .+:"
                # Find all matches
                matches = re.findall(pattern, message['content'])
                for match in matches:
                    message_to_print += match + " ...\n"
                
                print(f"{bold(message['role'].upper())}: {message_to_print}\n")

            else:
                print(f"{bold(message['role'].upper())}: {message['content']}\n")
        
        print(f"\n {bold('Costo total de la conversación'.upper())}: {self.total_cost:.7f}")

StatementMeta(spAzureML, 178, 37, Finished, Available, Finished)

In [21]:
# Solución: Copiar esto arriba

    def chat(self, message: str, print_conversation: bool = True):

        if self.db_search_done:
            self.process_new_message(message, print_conversation)
        else:
            query = self.prepare_message(message)
            self.process_new_message(query, print_conversation)

StatementMeta(spAzureML, 178, 38, Finished, Available, Finished)

IndentationError: unexpected indent (2751300339.py, line 3)

In [22]:
# Creo una instancia de la clase
rag_chatbot = RAGChatBot()

# Consulta del usuario
consulta = "Que requisitos documentales deben cumplir los vehículos que se utilizan?"

# Se realiza la consulta del usuario al modelo
rag_chatbot.chat(message=consulta)

StatementMeta(spAzureML, 178, 39, Finished, Available, Finished)

USER: La pregunta del usuario es: 'Que requisitos documentales deben cumplir los vehículos que se utilizan?'.
 La información para responder la pregunta es la que se indica a continuación: 
Capítulo 3 del documento P_LOGT_01: TRANSPORTE DE MADERA: ...
Capítulo 1 del documento P_LOGT_01: TRANSPORTE DE MADERA: ...
Capítulo 5 del documento P_LOGT_01: TRANSPORTE DE MADERA: ...


ASSISTANT: Según la información proporcionada en el documento P_LOGT_01: TRANSPORTE DE MADERA, los requisitos documentales que deben cumplir los vehículos que se utilizan para el transporte de madera son los siguientes:

Documentos para el Transportista (Empresa):
- Certificado de empresa profesional de carga
- Certificados, recibos y consulta de actividad por empresa al día en BPS
- Certificado de DGI vigente
- Seguro y certificados al día (BSE o Seguro alternativo, para accidentes laborales y contra Terceros)
- Planilla de MTSS

Documentos para el Transportista (Personal):
- Alta de cada funcionario emitida por B

In [23]:
# Continuo la conversación con el modelo
consulta = "Dice algo más el capítulo 3 acerca de los vehículos?"
rag_chatbot.chat(message=consulta)

StatementMeta(spAzureML, 178, 40, Finished, Available, Finished)

USER: La pregunta del usuario es: 'Que requisitos documentales deben cumplir los vehículos que se utilizan?'.
 La información para responder la pregunta es la que se indica a continuación: 
Capítulo 3 del documento P_LOGT_01: TRANSPORTE DE MADERA: ...
Capítulo 1 del documento P_LOGT_01: TRANSPORTE DE MADERA: ...
Capítulo 5 del documento P_LOGT_01: TRANSPORTE DE MADERA: ...


ASSISTANT: Según la información proporcionada en el documento P_LOGT_01: TRANSPORTE DE MADERA, los requisitos documentales que deben cumplir los vehículos que se utilizan para el transporte de madera son los siguientes:

Documentos para el Transportista (Empresa):
- Certificado de empresa profesional de carga
- Certificados, recibos y consulta de actividad por empresa al día en BPS
- Certificado de DGI vigente
- Seguro y certificados al día (BSE o Seguro alternativo, para accidentes laborales y contra Terceros)
- Planilla de MTSS

Documentos para el Transportista (Personal):
- Alta de cada funcionario emitida por B

# 4) Control de flujo

Vamos a crear una clase para llamar al modelo de Moderación. Este modelo sirve para controlar que los mensajes que recibe el chat tengan el tono y caracter adecuado para la situación.

Luego vamos a crear otra clase que sea la que dirija el flujo de la conversación. Esta clase va a controlar los mensajes que recibe el chat y si pasan la moderación, se continúa el flujo normal de la conversación. Si no pasa, se ignora la pregunta y se comunica al usuario que su mensaje no pasó las reglas de moderación.

In [16]:
class ModerationModel:
    def __init__(self, client):
        self.client = client

    def check_moderation(self, message: str) -> bool:
        response = self.client.moderations.create(input=message)
        result = response.results[0]

        # El modelo de moderación retorna `True` si el mensaje potencialmente contiene contenido no apto
        # Para retornar el valor, invertimos el resultado de la variable `.flagged` por comodidad
        return not result.flagged

StatementMeta(spAzureML, 164, 33, Finished, Available, Finished)

### Ejercicio
La clase `FlowChatBot` sobreescribe el método `chat` definido en clases de las que hereda. Hay que escribir la nueva implementación del método `chat` para que tenga en cuenta si el mensaje del usuario aprobó las reglas de moderación o no.

In [19]:
class FlowChatBot(RAGChatBot):
    def __init__(self, prompt: Union[str, None] = None, llm_name: Union[str, None] = None):
        super(FlowChatBot, self).__init__(prompt=prompt, llm_name=llm_name)
        self.moderation_model = ModerationModel(self.openai_client)
    
    # Sobreescribimos la clase chat heredada de RAGChatBot, que a su vez hereda de VanilaChatBot
    def chat(self, message: str, print_conversation: bool = True):

        moderation_succeeded = self.moderation_model.check_moderation(message)
        
        # ================================================================
        # Complete aquí su solución
        # ----------------------------------------------------------------
        
        moderation_succeeded = self.moderation_model.check_moderation(message)
        if moderation_succeeded:
            if self.db_search_done:
                self.process_new_message(message, print_conversation)
            else:
                query = self.prepare_message(message)
                self.process_new_message(query, print_conversation)
        else:
            print("Su mensaje no cumple las reglas de moderación. Chat terminado")
            
        # ================================================================
        


StatementMeta(spAzureML, 164, 36, Finished, Available, Finished)

In [ ]:
# Solución
    
    def chat(self, message: str, print_conversation: bool = True):

        moderation_succeeded = self.moderation_model.check_moderation(message)
        if moderation_succeeded:
            if self.db_search_done:
                self.process_new_message(message, print_conversation)
            else:
                query = self.prepare_message(message)
                self.process_new_message(query, print_conversation)
        else:
            print("Su mensaje no cumple las reglas de moderación. Chat terminado")

StatementMeta(, , , Waiting, , Waiting)

In [20]:
flow_chatbot = FlowChatBot()
flow_chatbot.chat("Los europeos son mucho mejores que los latinoamericanos?")

StatementMeta(spAzureML, 164, 37, Finished, Available, Finished)

Su mensaje no cumple las reglas de moderación. Chat terminado


In [21]:
# Consulta del usuario
consulta = "Que requisitos documentales deben cumplir los vehículos que se utilizan?"

# Se realiza la consulta del usuario al modelo
flow_chatbot.chat(message=consulta)

StatementMeta(spAzureML, 164, 38, Finished, Available, Finished)

USER: La pregunta del usuario es: 'Que requisitos documentales deben cumplir los vehículos que se utilizan?'.
 La información para responder la pregunta es la que se indica a continuación: 
Capítulo 3 del documento P_LOGT_01: TRANSPORTE DE MADERA: ...
Capítulo 1 del documento P_LOGT_01: TRANSPORTE DE MADERA: ...
Capítulo 5 del documento P_LOGT_01: TRANSPORTE DE MADERA: ...


ASSISTANT: Según la información proporcionada en el Capítulo 3 del documento P_LOGT_01: TRANSPORTE DE MADERA, los vehículos que se utilizan deben cumplir con los siguientes requisitos documentales:

- Inscripción en el registro de la Dirección Nacional de Transporte (MTOP)
- Inspección Técnica Vehicular (CAT)
- Permiso Nacional de Circulación
- Seguro de Responsabilidad Civil
- Libreta de propiedad
- Otras habilitaciones específicas del vehículo o carga

Además, los conductores de los vehículos deben contar con:
- Alta de cada funcionario emitida por BPS
- Cédula de identidad vigente
- Control de salud vigente
- Li

In [22]:
# Continuo la conversación con el modelo
consulta = "Hay alguna restricción acerca de qué tipos de vehículos se pueden utilizar?"
flow_chatbot.chat(message=consulta)

StatementMeta(spAzureML, 164, 39, Finished, Available, Finished)

USER: La pregunta del usuario es: 'Que requisitos documentales deben cumplir los vehículos que se utilizan?'.
 La información para responder la pregunta es la que se indica a continuación: 
Capítulo 3 del documento P_LOGT_01: TRANSPORTE DE MADERA: ...
Capítulo 1 del documento P_LOGT_01: TRANSPORTE DE MADERA: ...
Capítulo 5 del documento P_LOGT_01: TRANSPORTE DE MADERA: ...


ASSISTANT: Según la información proporcionada en el Capítulo 3 del documento P_LOGT_01: TRANSPORTE DE MADERA, los vehículos que se utilizan deben cumplir con los siguientes requisitos documentales:

- Inscripción en el registro de la Dirección Nacional de Transporte (MTOP)
- Inspección Técnica Vehicular (CAT)
- Permiso Nacional de Circulación
- Seguro de Responsabilidad Civil
- Libreta de propiedad
- Otras habilitaciones específicas del vehículo o carga

Además, los conductores de los vehículos deben contar con:
- Alta de cada funcionario emitida por BPS
- Cédula de identidad vigente
- Control de salud vigente
- Li

# 5) Nueva funcionalidad: resumen de documentos

El flujo de conversación puede ir adquiriendo mayor complejidad a medida que se van agregando nuevas funcionalidades al `ChatBot`. 

En esta sección vamos a ver otra tarea que se le puede pedir al ChatBot, la tarea de resumir documentos. Para ello vamos a crear una nueva clase `SummaryChatBot`, que hereda de `RAGChatBot`. 

Vamos a suponer que el mensaje que le pasa el usuario cuando quiere resumir un texto es el nombre del documento que quiere resumir. Luego el usuario puede continuar interactuando con el modelo preguntandole información acerca de ese mismo documento. 

In [28]:
class SummaryChatBot(RAGChatBot):
    def __init__(self, prompt: Union[str, None] = None, llm_name: Union[str, None] = None):
        
        if not prompt:
            prompt = """
                El usuario te brindará información técnica que quiere resumir. 
                Elaborar un resumen en base a esa información, que no supere las 1000 palabras. 
                Luego te hará preguntas en base a esa información. 
                En caso que te hagan una pregunta cuya respuesta no quede comprendida en la información provista, 
                responder que no contas con información para responder la pregunta. 
                Responder siempre en español. 
                Al responder indicar de donde extrajiste la información para elaborar la respuesta. 
            """   
        super(SummaryChatBot, self).__init__(prompt=prompt, llm_name=llm_name)

    # Método donde se arma el mensaje con todos los documentos para consultarle al modelo
    # SUPONEMOS QUE EL MENSAJE INDICADO POR EL USUARIO ES EL NOMBRE DEL DOCUMENTO QUE QUIERE RESUMIR   
    def prepare_message(self, message: str) -> str:
        
        condition = {
            "nombre_documento": {
                "$eq": message
            }
        }

        df_chroma_results = self.chroma_db.db_search(condition=condition)
        df_chroma_results_sorted = df_chroma_results.sort_values(by='id', key=lambda x: x.astype(int))
        self.db_search_done = True

        doc_name = df_chroma_results_sorted.iloc[0]['nombre_documento']
        query = "La instrucción del usuario es: '" + message + "'.\n La información que se debe resumir"\
        " se presenta a continuación, y proviene del documento " + doc_name + ". \n"
        self.query_header = query

        # Agregamos documentos al mensaje
        for i in range(len(df_chroma_results_sorted)):
            row = df_chroma_results_sorted.iloc[i]
            query += "Capítulo "+row['capitulo']+": "\
            + row['contenido'] + "\n"
        return query
    
    # Sobrescribimos el método print_history para no mostrar todo el contenido de los documentos que
    # le estamos pasando al modelo
    def print_history(self):
        def bold(text: str):
            return '\033[1m'+ text +'\033[0m'

        for message in self.history[1:]:
            if message['role'] == 'user' and self.query_header in message['content']:
                message_to_print = self.query_header
                # Regular expression pattern
                pattern = r"Capítulo \d+:"
                # Find all matches
                matches = re.findall(pattern, message['content'])
                for match in matches:
                    message_to_print += match + " ...\n"
                
                print(f"{bold(message['role'].upper())}: {message_to_print}\n")

            else:
                print(f"{bold(message['role'].upper())}: {message['content']}\n")
        
        print(f"\n {bold('Costo total de la conversación'.upper())}: {self.total_cost:.7f}")

StatementMeta(spAzureML, 164, 45, Finished, Available, Finished)

In [29]:
# Creo una instancia de la clase
summary_chatbot = SummaryChatBot()

# Consulta del usuario
consulta = "P_LOGT_01: TRANSPORTE DE MADERA"

# Se realiza la consulta del usuario al modelo
summary_chatbot.chat(message=consulta)

StatementMeta(spAzureML, 164, 46, Finished, Available, Finished)

USER: La instrucción del usuario es: 'P_LOGT_01: TRANSPORTE DE MADERA'.
 La información que se debe resumir se presenta a continuación, y proviene del documento P_LOGT_01: TRANSPORTE DE MADERA. 
Capítulo 0: ...
Capítulo 1: ...
Capítulo 2: ...
Capítulo 3: ...
Capítulo 4: ...
Capítulo 5: ...
Capítulo 6: ...
Capítulo 7: ...
Capítulo 8: ...
Capítulo 9: ...
Capítulo 10: ...
Capítulo 11: ...


ASSISTANT: El documento "P_LOGT_01: TRANSPORTE DE MADERA" establece los lineamientos y procedimientos para garantizar un transporte eficiente y seguro de madera rolliza, respetando aspectos sociales y ambientales. El objetivo es planificar, habilitar conductores y vehículos, cargar la madera, controlar la operación, seguir medidas de seguridad, prevenir riesgos laborales e impactos ambientales, y establecer responsabilidades y registros.

En cuanto a los lineamientos generales, se establece la planificación del transporte a largo plazo, a 5 años y mensualmente, y se detallan los requerimientos document

In [ ]:
# Continuo la conversación con el modelo
consulta = "Qué dice el documento que debe hacer el transportista en caso que se presente una falla en el vehículo?"
summary_chatbot.chat(message=consulta)

StatementMeta(, , , Waiting, , Waiting)